# Title

## Setup and Helper Functions

In [ ]:
import json
from pathlib import Path

import matplotlib.pyplot as plt
import seaborn as sns
from fau_colors import cmaps, register_fausans_font

from pepbench.datasets import EmpkinsDataset, GuardianDataset
from pepbench.evaluation import PepEvaluationChallenge

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"

palette

In [ ]:
deploy_type = "local"

config_dict = json.load(Path("./config.json").open(encoding="utf-8"))

guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
guardian_base_path

## Guardian Dataset

In [ ]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

In [ ]:
subset = dataset_guardian.get_subset(participant="GDN0005", phase="Pause")

labeling_borders = subset.labeling_borders
reference_labels_icg = subset.reference_labels_icg
reference_labels_ecg = subset.reference_labels_ecg
ecg_data = subset.ecg
icg_data = subset.icg

display(labeling_borders)
display(reference_labels_ecg)
display(ecg_data)

## EmpkinS Dataset

In [ ]:
empkins_base_path = Path(config_dict[deploy_type]["empkins_path"])
empkins_base_path

In [ ]:
dataset_empkins = EmpkinsDataset(empkins_base_path, use_cache=False, only_labeled=True)
dataset_empkins

In [ ]:
subset = dataset_empkins.get_subset(participant="VP_001", condition="tsst", phase="Math")

labeling_borders = subset.labeling_borders
reference_labels_icg = subset.reference_labels_icg
reference_labels_ecg = subset.reference_labels_ecg
ecg_data = subset.ecg
icg_data = subset.icg

display(labeling_borders)
display(reference_labels_ecg)
display(ecg_data)

## Plot Tests

In [ ]:
reference_labels_ecg

In [ ]:
# fig, ax = plt.subplots()
#
# subset = dataset_empkins.get_subset(participant="VP_001", condition="tsst", phase="Pause_1")
#
# ecg_data = subset.ecg
# reference_labels_ecg = subset.reference_labels_ecg
# labeling_borders = subset.labeling_borders
#
# start_end_borders = reference_labels_ecg.xs("heartbeat", level="channel")
# qwave_labels = reference_labels_ecg.xs("Q-wave_onset", level="label")
#
# ecg_data.plot(ax=ax)
#
# ax.vlines(x=ecg_data.index[start_end_borders["sample_relative"]], ymin=0, ymax=1, colors=cmaps.tech[2], transform=ax.get_xaxis_transform())
# ax.scatter(x=ecg_data.index[qwave_labels["sample_relative"]], y=ecg_data["ecg"][ecg_data.index[qwave_labels["sample_relative"]]], color=cmaps.nat[0], zorder=3)
#
# fig.tight_layout()

## Signal Processing Tests

In [ ]:
# subset = dataset_empkins.get_subset(participant="VP_001", condition="tsst", phase="Math")
# icg_data = subset.icg

# fig, ax = plt.subplots()
# icg_data.plot(ax=ax)
# fig.tight_layout()

In [ ]:
subset = dataset_empkins.get_subset(participant="VP_001", condition="tsst", phase="Math")
ecg_data = subset.ecg

fig, ax = plt.subplots()
ecg_data.plot(ax=ax)
fig.tight_layout()

In [ ]:
subset = dataset_empkins.get_subset(participant="VP_001", condition="tsst", phase="Pause_1")

In [ ]:
subset = dataset_guardian.get_subset(participant="GDN0005", phase="Pause")
# subset.reference_pep

## Algorithm Tests

In [ ]:
from biopsykit.signals.ecg.event_extraction import (
    QPeakExtractionVanLien2013,
)
from biopsykit.signals.ecg.segmentation import HeartbeatSegmentationNeurokit
from biopsykit.signals.icg.event_extraction import (
    BPointExtractionForouzanfar2018,
    CPointExtractionScipyFindPeaks,
)
from biopsykit.signals.icg.outlier_correction import (
    OutlierCorrectionForouzanfar2018,
    OutlierCorrectionLinearInterpolation,
)

In [ ]:
# subset = dataset_empkins.get_subset(participant="VP_001", condition="tsst", phase="Pause_1")
subset = dataset_guardian.get_subset(participant="GDN0005", phase="Pause")

In [ ]:
ecg = subset.ecg
icg = subset.icg
fs = subset.sampling_rate_ecg

In [ ]:
fig, ax = plt.subplots()
icg.plot(ax=ax)

In [ ]:
heartbeats = subset.heartbeats
heartbeats

In [ ]:
ecg_extraction = QPeakExtractionVanLien2013(time_interval_ms=40, handle_missing_events="ignore")
ecg_extraction.extract(ecg=ecg, heartbeats=heartbeats, sampling_rate_hz=fs)
ecg_extraction.points_

In [ ]:
heartbeat_segmentation = HeartbeatSegmentationNeurokit()

cpoint_extraction = CPointExtractionScipyFindPeaks()
cpoint_extraction.extract(icg=icg, heartbeats=heartbeats, sampling_rate_hz=fs)

display(cpoint_extraction.points_)

bpoint_extraction = BPointExtractionForouzanfar2018(correct_outliers=True)
bpoint_extraction.extract(
    icg=icg, heartbeats=heartbeats, c_points=cpoint_extraction.points_, sampling_rate_hz=fs
)
display(bpoint_extraction.points_)

outlier_correction = OutlierCorrectionLinearInterpolation()
outlier_correction.correct_outlier(
    b_points=bpoint_extraction.points_, c_points=cpoint_extraction.points_, sampling_rate_hz=fs
)
outlier_correction.points_

In [ ]:
from pepbench.pipelines import (
    PepExtractionPipeline,
    PepExtractionPipelineReferenceBPoints,
    PepExtractionPipelineReferenceQPeaks,
)

In [ ]:
heartbeat_segmentation_algo = HeartbeatSegmentationNeurokit()
q_wave_algo = QPeakExtractionVanLien2013()
b_point_algo = BPointExtractionForouzanfar2018(correct_outliers=True)
outlier_correction_algo = OutlierCorrectionForouzanfar2018()
# outlier_correction_algo = OutlierCorrectionDummy()

In [ ]:
pipeline_qpoints = PepExtractionPipelineReferenceQPeaks(
    heartbeat_segmentation_algo=heartbeat_segmentation_algo,
    q_peak_algo=q_wave_algo,
    b_point_algo=b_point_algo,
    outlier_correction_algo=outlier_correction_algo,
    handle_missing_events="ignore",
)

In [ ]:
pipeline_bpoints = PepExtractionPipelineReferenceBPoints(
    heartbeat_segmentation_algo=heartbeat_segmentation,
    q_peak_algo=q_wave_algo,
    b_point_algo=b_point_algo,
    outlier_correction_algo=outlier_correction_algo,
    handle_missing_events="ignore",
)

In [ ]:
pipeline = PepExtractionPipeline(
    heartbeat_segmentation_algo=heartbeat_segmentation_algo,
    q_peak_algo=q_wave_algo,
    b_point_algo=b_point_algo,
    outlier_correction_algo=outlier_correction_algo,
    handle_negative_pep="nan",
    handle_missing_events="ignore",
)

In [ ]:
subset_guardian = dataset_guardian.get_subset(participant=["GDN0005", "GDN0006"])
pep_challenge = PepEvaluationChallenge(dataset=subset_guardian)

pep_challenge.run(pipeline)

In [ ]:
pep_challenge.results_as_df()

In [ ]:
pep_challenge.results_agg_mean_std_

In [ ]:
# subset_test = dataset_empkins.get_subset(participant="VP_002", condition="ftsst", phase="Talk")
# match_heartbeat_lists(heartbeats_reference=subset_test.reference_heartbeats, heartbeats_extracted=subset_test.heartbeats, tolerance_ms=20, sampling_rate_hz=subset_test.sampling_rate_ecg)

In [ ]:
fig, ax = plt.subplots()

icg_data = subset.icg
reference_labels_icg = subset.reference_labels_icg
labeling_borders = subset.labeling_borders

start_end_borders = reference_labels_icg.xs("heartbeat", level="channel")
bpoint_labels = reference_labels_icg.xs("B-point", level="label")
# bpoint_artefacts = reference_labels_icg.xs("Artefact", level="label")
bpoint_labels_algo = bpoint_extraction.points_["b_point_sample"].dropna()
bpoint_labels_outlier = outlier_correction.points_["b_point_sample"].dropna()

icg_data.plot(ax=ax)

ax.vlines(
    x=icg_data.index[start_end_borders["sample_relative"]],
    ymin=0,
    ymax=1,
    colors=cmaps.tech[2],
    transform=ax.get_xaxis_transform(),
)
ax.scatter(
    x=icg_data.index[bpoint_labels["sample_relative"]],
    y=icg_data["icg_der"][icg_data.index[bpoint_labels["sample_relative"]]],
    color=cmaps.nat[0],
    zorder=3,
    label="Reference",
)
# ax.scatter(x=icg_data.index[bpoint_artefacts["sample_relative"]], y=icg_data["icg_der"][icg_data.index[bpoint_artefacts["sample_relative"]]], color=cmaps.med[0], zorder=3, label="Artefacts")
ax.scatter(
    x=icg_data.index[bpoint_labels_algo.astype(int)],
    y=icg_data["icg_der"][icg_data.index[bpoint_labels_algo.astype(int)]],
    color=cmaps.phil[0],
    zorder=3,
    label="Algorithm",
)
ax.scatter(
    x=icg_data.index[bpoint_labels_outlier.astype(int)],
    y=icg_data["icg_der"][icg_data.index[bpoint_labels_outlier.astype(int)]],
    color=cmaps.wiso[0],
    zorder=3,
    label="Outlier Correction",
)

ax.legend()

fig.tight_layout()

In [ ]:
fig, ax = plt.subplots()

ecg_data = subset.ecg
reference_labels_ecg = subset.reference_labels_ecg
labeling_borders = subset.labeling_borders

start_end_borders = reference_labels_ecg.xs("heartbeat", level="channel")
qwave_labels = reference_labels_ecg.xs("Q-wave_onset", level="label")
qwave_labels_algo = ecg_extraction.points_["q_peak_sample"].dropna()

ecg_data.plot(ax=ax)

ax.vlines(
    x=ecg_data.index[start_end_borders["sample_relative"]],
    ymin=0,
    ymax=1,
    colors=cmaps.tech[2],
    transform=ax.get_xaxis_transform(),
)
ax.scatter(
    x=ecg_data.index[qwave_labels["sample_relative"]],
    y=ecg_data["ecg"][ecg_data.index[qwave_labels["sample_relative"]]],
    color=cmaps.nat[0],
    zorder=3,
)
ax.scatter(
    x=ecg_data.index[qwave_labels_algo.astype(int)],
    y=ecg_data["ecg"][ecg_data.index[qwave_labels_algo.astype(int)]],
    color=cmaps.phil[0],
    zorder=3,
)

fig.tight_layout()